# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

# Train

In [2]:
train_set = pd.read_csv('train/train_encoded.csv')

In [3]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [4]:
batch_size = 1
num_classes = 2
epochs = 10

In [5]:
X_train

,len_location,len_keyword,len_text,len_text_original,diff_len_text,keyword_value,keyword_sum,keyword_min,keyword_max,keyword_mean,...,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_value,text_norm_value,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v
7330,11,7,82,140,58,4.268695,1.000000,1.000000,1.000000,1.000000,...,0.386228,0.252131,0.247074,46.484588,17.931932,19.229431,-0.115915,0.568572,0.112453,0.093215
6601,0,6,87,140,53,3.397147,1.000000,1.000000,1.000000,1.000000,...,0.543414,0.288275,0.271707,39.706227,18.755461,13.829157,-0.039208,1.000000,0.151969,0.105314
5834,11,5,48,92,44,4.130719,1.000000,1.000000,1.000000,1.000000,...,0.403980,0.320793,0.363643,34.642470,14.492268,11.169735,0.000892,0.542930,0.122744,0.095739
6171,30,5,16,26,10,3.827693,1.000000,1.000000,1.000000,1.000000,...,0.656702,0.574148,0.553355,11.499934,6.794611,0.511770,0.004571,0.159618,0.085295,0.090074
2317,9,8,70,138,68,3.226890,1.000000,1.000000,1.000000,1.000000,...,0.423989,0.295990,0.327350,35.721459,16.136969,26.780494,-0.057497,1.000000,0.196915,0.146862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3152,8,5,72,137,65,2.949969,1.000000,1.000000,1.000000,1.000000,...,0.412201,0.286840,0.283988,20.879887,9.780977,2.718876,-0.103342,1.000000,0.097103,0.041653
575,0,9,76,138,62,4.219201,1.000000,1.000000,1.000000,1.000000,...,0.439925,0.294547,0.322111,30.273171,14.194214,7.353818,-0.030709,1.000000,0.133706,0.106252
3195,23,10,97,132,35,5.290447,1.403003,0.612643,0.790360,0.701501,...,0.387993,0.269390,0.232981,34.803419,15.609525,12.383357,-0.048282,0.634660,0.136081,0.108446
1184,23,13,50,83,33,6.173725,1.408101,0.638380,0.769722,0.704051,...,0.417117,0.320506,0.352132,23.781937,10.863061,3.762588,-0.026625,0.312744,0.104516,0.090719


In [6]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [7]:
X_train.shape

(5329, 28, 1)

In [8]:
input_shape = (X_train.shape[1], 1)
input_shape

(28, 1)

## Relu

In [9]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 27, 128)           384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 64)            8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 9, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 9, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               73856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [11]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.5609 - accuracy: 0.5138 - val_loss: 6.5056 - val_accuracy: 0.5749
Epoch 2/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.5199 - accuracy: 0.5258 - val_loss: 6.5057 - val_accuracy: 0.5753
Epoch 3/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.6076 - accuracy: 0.5142 - val_loss: 6.5082 - val_accuracy: 0.5749
Epoch 4/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.5078 - accuracy: 0.5267 - val_loss: 6.5082 - val_accuracy: 0.5753
Epoch 5/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.3880 - accuracy: 0.5243 - val_loss: 6.5083 - val_accuracy: 0.5762


In [12]:
model = Sequential()
model.add(Conv1D(128, kernel_size=3,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 26, 128)           512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 6, 128)            16512     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 6, 32)             4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3, 32)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 3, 64)            

In [14]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 12s 2ms/step - loss: 7.2235 - accuracy: 0.5078 - val_loss: 7.7748 - val_accuracy: 0.4256
Epoch 2/5
5329/5329 [==============================] - 12s 2ms/step - loss: 6.9102 - accuracy: 0.4937 - val_loss: 7.2444 - val_accuracy: 0.4251
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 6.7238 - accuracy: 0.5200 - val_loss: 6.5217 - val_accuracy: 0.4282
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.5838 - accuracy: 0.5042 - val_loss: 5.0621 - val_accuracy: 0.4234
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.4116 - accuracy: 0.5192 - val_loss: 4.4729 - val_accuracy: 0.4260


In [15]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=10))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 28, 1024)          2048      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 1024)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 1024)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 2, 512)            524800    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 2, 32)             16416     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2, 216)           

In [17]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 42s 8ms/step - loss: 3.2425 - accuracy: 0.5296 - val_loss: 0.6812 - val_accuracy: 0.5792
Epoch 2/5
5329/5329 [==============================] - 46s 9ms/step - loss: 2.6731 - accuracy: 0.5179 - val_loss: 0.6830 - val_accuracy: 0.4221
Epoch 3/5
5329/5329 [==============================] - 45s 8ms/step - loss: 2.4629 - accuracy: 0.4918 - val_loss: 0.6894 - val_accuracy: 0.4221
Epoch 4/5
5329/5329 [==============================] - 55s 10ms/step - loss: 2.1333 - accuracy: 0.4900 - val_loss: 0.6900 - val_accuracy: 0.4221
Epoch 5/5
5329/5329 [==============================] - 40s 8ms/step - loss: 1.7980 - accuracy: 0.5100 - val_loss: 0.6861 - val_accuracy: 0.4221


## Softmax

In [18]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.5459 - val_loss: 0.6932 - val_accuracy: 0.5679
Epoch 2/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.5303 - val_loss: 0.6931 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 20s 4ms/step - loss: 0.6932 - accuracy: 0.5207 - val_loss: 0.6931 - val_accuracy: 0.5679
Epoch 4/5
5329/5329 [==============================] - 21s 4ms/step - loss: 0.6932 - accuracy: 0.5080 - val_loss: 0.6931 - val_accuracy: 0.5679
Epoch 5/5
5329/5329 [==============================] - 20s 4ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.5679


## Tanh

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 19s 4ms/step - loss: 4.3310 - accuracy: 0.5528 - val_loss: 4.0523 - val_accuracy: 0.5679
Epoch 2/5
5329/5329 [==============================] - 15s 3ms/step - loss: 4.1956 - accuracy: 0.5393 - val_loss: 3.9104 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 15s 3ms/step - loss: 4.0135 - accuracy: 0.5472 - val_loss: 3.8582 - val_accuracy: 0.5679
Epoch 4/5
5329/5329 [==============================] - 14s 3ms/step - loss: 3.7964 - accuracy: 0.5361 - val_loss: 3.8736 - val_accuracy: 0.5679
Epoch 5/5
5329/5329 [==============================] - 22s 4ms/step - loss: 3.4858 - accuracy: 0.5239 - val_loss: 3.1945 - val_accuracy: 0.5683


# Mix

In [22]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 25, 128)           256       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 8, 128)            0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 8, 128)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 8, 128)            16512     
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 8, 32)             4128      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 2, 32)             0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 2, 64)            

In [24]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.4468 - accuracy: 0.4425 - val_loss: 7.6024 - val_accuracy: 0.4321
Epoch 2/5
5329/5329 [==============================] - 16s 3ms/step - loss: 7.4820 - accuracy: 0.4483 - val_loss: 7.6024 - val_accuracy: 0.4321
Epoch 3/5
5329/5329 [==============================] - 18s 3ms/step - loss: 7.3524 - accuracy: 0.4445 - val_loss: 7.6021 - val_accuracy: 0.4321
Epoch 4/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.3611 - accuracy: 0.4474 - val_loss: 7.5996 - val_accuracy: 0.4321
Epoch 5/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.2968 - accuracy: 0.4421 - val_loss: 7.5993 - val_accuracy: 0.4321


## Text w2v

In [3]:
train_set = pd.read_csv('train/text_encoded_w2v.csv')

In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [5]:
batch_size = 1
num_classes = 2
epochs = 10

In [6]:
X_train

,id,text_sum,text_min,text_max,text_mean,text_median
5535,7895,1.885115,-0.031004,0.262091,0.089767,0.059510
6018,8596,9.313427,-0.040400,1.000000,0.102345,0.069088
4739,6738,14.726642,0.012219,1.000000,0.188803,0.132279
3181,4567,3.985746,-0.040585,0.433890,0.110715,0.082433
818,1189,3.576633,-0.090845,0.443895,0.170316,0.168023
...,...,...,...,...,...,...
5282,7548,2.514225,-0.025919,1.000000,0.119725,0.074594
563,814,4.079575,-0.013456,1.000000,0.271972,0.073396
2089,3003,12.973744,-0.108944,1.000000,0.142569,0.114495
3221,4622,3.870319,-0.074206,0.450665,0.107509,0.085664


In [7]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [8]:
X_train.shape

(5329, 6, 1)

In [9]:
input_shape = (X_train.shape[1], 1)
input_shape

(6, 1)

## Relu

In [10]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 5, 128)            384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 64)             8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [12]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 8.2134 - accuracy: 0.4460 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 8.2596 - accuracy: 0.4466 - val_loss: 8.4191 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.1505 - accuracy: 0.4399 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2660 - accuracy: 0.4384 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2680 - accuracy: 0.4372 - val_loss: 8.4216 - val_accuracy: 0.4435


In [14]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 6, 128)            256       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 6, 128)            16512     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 32)             4128      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 6, 64)            

In [16]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.9467 - accuracy: 0.5384 - val_loss: 8.3027 - val_accuracy: 0.5565
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.9275 - accuracy: 0.5485 - val_loss: 8.2378 - val_accuracy: 0.5565
Epoch 3/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.8626 - accuracy: 0.5372 - val_loss: 8.1313 - val_accuracy: 0.5565
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8740 - accuracy: 0.5393 - val_loss: 7.7117 - val_accuracy: 0.5565
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8631 - accuracy: 0.5378 - val_loss: 7.4220 - val_accuracy: 0.5565


In [17]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 6, 1024)           2048      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 6, 1024)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 1024)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 6, 512)            524800    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 6, 32)             16416     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 6, 216)           

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 47s 9ms/step - loss: 8.0898 - accuracy: 0.4265 - val_loss: 7.5757 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 51s 10ms/step - loss: 7.9531 - accuracy: 0.4260 - val_loss: 7.6082 - val_accuracy: 0.4426
Epoch 3/5
5329/5329 [==============================] - 50s 9ms/step - loss: 7.8164 - accuracy: 0.4218 - val_loss: 7.6192 - val_accuracy: 0.4426
Epoch 4/5
5329/5329 [==============================] - 47s 9ms/step - loss: 7.7759 - accuracy: 0.4256 - val_loss: 7.6203 - val_accuracy: 0.4431
Epoch 5/5
5329/5329 [==============================] - 48s 9ms/step - loss: 7.6847 - accuracy: 0.4237 - val_loss: 7.6232 - val_accuracy: 0.4431


## Softmax

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 0.6932 - accuracy: 0.4915 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 11s 2ms/step - loss: 0.6932 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4864 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.4435


## Tanh

In [22]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.2916 - accuracy: 0.4843 - val_loss: 0.8663 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.1247 - accuracy: 0.4881 - val_loss: 0.8411 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.1101 - accuracy: 0.4935 - val_loss: 0.8166 - val_accuracy: 0.4448
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.0732 - accuracy: 0.4817 - val_loss: 0.8006 - val_accuracy: 0.4453
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 2.9780 - accuracy: 0.5044 - val_loss: 0.7842 - val_accuracy: 0.4448
